In [1]:
import argparse
import logging
from operator import mul
import time
import os

import pubweb.singlecell # import AnnDataSparse
from pubweb.hdf5 import Hdf5
from pubweb.commands.convert.singlecell.anndata import ImportAnndata
from pubweb.commands.convert.singlecell.cellranger import ImportCellRanger
from pubweb.commands.validate.dimensions import ValidateDimensions
from pubweb.commands.annotate.geneid import AnnotateGeneId
from pubweb.commands.annotate.geneset import AnnotateGeneset
from pubweb.commands.export.lists import ExportLists
from pubweb.commands.export.attributes import ExportAttributes
from pubweb.commands.export.tables import ExportTables
from pubweb.commands.export.projections import ExportProjections
from pubweb.commands.export.spatial import ExportSpatial
from pubweb.commands.export.matrix_sparse import ExportMatrixSparse
from pubweb.commands.export.matrix_dense import ExportMatrixDense
from pubweb.commands.summarize.genes import SummarizeGenes
from pubweb.commands.summarize.genemap import SummarizeGeneMap
from pubweb.commands.summarize.colors import SummarizeColors
from pubweb.commands.summarize.manifest import SummerizeManifest


In [2]:
import requests
import os
import h5py
import pandas
import numpy as np
import boto3
import re
from io import BytesIO
from urllib.parse import urlparse
from shutil import copyfile
from pathlib import Path
from pubweb.enums import Attribute
from pubweb.hdf5 import Hdf5
from pubweb.commands.command import Command
import logging


In [3]:
import anndata

In [4]:
logging.basicConfig(level='DEBUG')

In [4]:
datasetName='skin-fake-h1299'
fileToConvert = '/data/notebooks/input3/input.h5ad'
inputFile = '/data/notebooks/input3/convert.hdf5'
outputFolder = '/data/notebooks/pubweb3'
species = 'human'
overwriteHdf5 = True
python_wd = '/opt/pubweb'

In [5]:
adata = anndata.read_h5ad(fileToConvert, backed='r')
adata.write(inputFile)

/root/anaconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])


In [6]:
with h5py.File(inputFile, "r+") as data:
    if 'index' in data['var'].keys():
        print('Moving /var/index group')
        data.move('var/index', 'var/_index')
    if 'index' in data['obs'].keys():
        print('Moving /obs/index group')
        data.move('obs/index', 'obs/_index')
    if 'shape' not in data['X'].attrs.keys():
        print("Adding shape to data['X'] b/c it's missing")
        data['X'].attrs['shape'] = [ data['X'].shape[0], data['X'].shape[1] ]

Moving /var/index group
Moving /obs/index group


In [8]:
# anndatasparse
outputFile = f'{outputFolder}/pubweb.hdf5'
if os.path.exists(outputFile) and overwriteHdf5:
    os.remove(outputFile)
hdf5 = Hdf5.load(outputFile, "a")

### data['X']

In [ ]:
data['X'].keys()
# 3 members, ['data', 'indices', 'indptr']


In [15]:
data = h5py.File(inputFile, 'r')

In [16]:
dataset = f'/pubweb/{datasetName}'



In [17]:
# group = hdf5.h5py.create_group(f'{dataset}')
group = hdf5.h5py[dataset]

In [ ]:
#data.copy('X', group, name="matrix")

In [ ]:
hdf5.h5py.create_dataset(f'testme', data=data['X'].attrs["shape"])
# <HDF5 dataset "testme": shape (2,), type "<i8">

In [22]:
group

<HDF5 group "/pubweb/lung-upper-airway-h1299" (1 members)>

### data['var']

In [ ]:
data['var'].keys()
# looks like it's a dataset of tuples. Each tuple

In [ ]:
# variables/features
# var is a dataset, not a group
# yep, 'FAM138A' is a gene

#data.move('')
#data['var'][()]

In [ ]:
type(data['var'])

In [ ]:
#

### data['obs']

In [ ]:
data['obs'].keys()

### data['obsm']

In [ ]:
data['obsm'].keys()

In [9]:
%time hdf5 | ImportAnndata(inputFile, datasetName)

RuntimeError: Unable to create link (message type not found)